In [1]:
# !pip install olefile

In [2]:
import olefile
import zlib
import struct

def get_hwp_text(filename):
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
       ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section"+str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed:
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data
    
        # 각 Section 내 text 추출    
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67]:
                rec_data = unpacked_data[i+4:i+4+rec_len]
                section_text += rec_data.decode('utf-16')
                section_text += "\n"

            i += 4 + rec_len

        text += section_text
        text += "\n"

    return text

In [3]:
import os
pathlist = ["data/raw/law"]

res = []
for path in pathlist:
    for root, dirs, files in os.walk(path):
        for file in files:
            filepath = os.path.join(root, file)
            res.append(filepath) 

In [4]:
res

['data/raw/law\\가등기담보 등에 관한 법률(법률)(제14474호)(20170328).hwp',
 'data/raw/law\\민법(인용조문 3단비교).hwp',
 'data/raw/law\\민사소송법(인용조문 3단비교).hwp',
 'data/raw/law\\민사집행법(인용조문 3단비교).hwp',
 'data/raw/law\\부동산 실권리자명의 등기에 관한 법률(인용조문 3단비교).hwp',
 'data/raw/law\\부동산등기법(인용조문 3단비교).hwp',
 'data/raw/law\\주민등록법(인용조문 3단비교).hwp',
 'data/raw/law\\주택임대차보호법(인용조문 3단비교).hwp',
 'data/raw/law\\지방자치법(인용조문 3단비교).hwp',
 'data/raw/law\\집합건물의 소유 및 관리에 관한 법률(인용조문 3단비교).hwp',
 'data/raw/law\\채무자 회생 및 파산에 관한 법률(인용조문 3단비교).hwp']

In [5]:
res = res[:2]
res

['data/raw/law\\가등기담보 등에 관한 법률(법률)(제14474호)(20170328).hwp',
 'data/raw/law\\민법(인용조문 3단비교).hwp']

In [6]:
import pandas as pd

def hwp_to_df(res):
    temp = []
    for i in range(len(res)):
        text = get_hwp_text(res[i])
        temp += text.split("\r\n")
    df = pd.DataFrame(temp, columns=["law"])
    return df

text = hwp_to_df(res)
text

,law
0,捤獥    汤捯     漠杳     浫╢    
1,가등기담보 등에 관한 법률 ( 약칭: 가등기담보법 )浫ɢ    
2,"慤桥    潴景    [시행 2017. 3. 28.] [법률 제14474호,..."
3,氠瑢    
4,가등기담보 등에 관한 법률
...,...
2329,[본조신설 1977. 12. 31.]
2330,제1116조(반환의 순서) 증여에 대하여는 유증을 반환받은 후가 아니면 이것을 청구...
2331,제1117조(소멸시효) 반환의 청구권은 유류분권리자가 상속의 개시와 반환하여야 할 ...
2332,"제1118조(준용규정) 제1001조, 제1008조, 제1010조의 규정은 유류분에 ..."


In [7]:
text.to_csv("data/law_preprocess.csv", index=False)